In [1]:
cd ..

/Users/johanbarreiro/Documents/GitHub/vl_optimizer


In [2]:
import pandas as pd
import os
import data.data_utils as du

def join_csv_on_time_column(directory='.'):
    # Get all CSV files in the directory
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    
    # Initialize an empty dataframe
    merged_df = pd.DataFrame()
    
    # Iterate through each CSV file and merge them on the 'Time' column
    for csv_file in csv_files:
        df = pd.read_csv(os.path.join(directory, csv_file))
        
        df = df[['Time', 'Forecasting Values']]
        
        column_name = csv_file[:-21]
        
        df[column_name] = df['Forecasting Values']
        
        df = df.drop('Forecasting Values', axis=1)  
        
        # Check if 'Time' column exists in the dataframe
        if 'Time' not in df.columns:
            print(f"'Time' column not found in {csv_file}")
            continue
        
        # Merge the dataframes
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='Time', how='outer')
    
    return merged_df

# Run the function and store the result
merged_dataframe = join_csv_on_time_column(directory='data/forecasted_data')



In [3]:
processed_df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-07T16-56-34_industrial_site2_processed.csv')

In [4]:
merged_dataframe = pd.merge(merged_dataframe, processed_df[['Time', 'heat_index', 'price_mWh']], on='Time', how='inner')

In [5]:
merged_dataframe['Hour'] = merged_dataframe['Time'].apply(lambda x: int(x.split(' ')[1].split(':')[0]))

In [6]:
merged_dataframe['efficiency_adjusted_power'] = merged_dataframe['chiller_efficiency']
merged_dataframe = merged_dataframe.drop('chiller_efficiency', axis=1)

merged_dataframe['technological_centers_consumption'] = merged_dataframe['08 Technological Centers_Electric_Active Energy (kWh)']
merged_dataframe = merged_dataframe.drop('08 Technological Centers_Electric_Active Energy (kWh)', axis=1)

In [7]:
merged_dataframe.isnull().sum().sum(axis = 0)

6

In [8]:
merged_dataframe  = merged_dataframe.interpolate(method='linear', axis=0)

/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/ipykernel_36924/2630850433.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  merged_dataframe  = merged_dataframe.interpolate(method='linear', axis=0)


In [9]:
merged_dataframe.isnull().sum().sum(axis = 0)

0

In [10]:
merged_dataframe.head(30)

,Time,production_schedule,compressor_consumption,operational_presence,testing_schedule,active_computers,maintenance_status,lightbulbs_active,workload,volume_production_waste,fabric_in_chamber,active_wall_plugs,number_of_workers,heat_index,price_mWh,Hour,efficiency_adjusted_power,technological_centers_consumption
0,2023-06-01 00:00:00,0,1.008715,0.000000,0.091031,0,0,0,0.000000,0,0.000000,0,0,14.27,94.09,0,0.608466,1.001112
1,2023-06-01 01:00:00,31,16.339983,4.370405,15.624779,1,15,1,23.990761,17,11.078917,2,0,13.77,87.00,1,0.609133,30.909301
2,2023-06-01 02:00:00,0,16.491206,3.889101,17.954943,1,24,1,25.457470,15,3.723168,0,73,13.60,87.00,2,0.609876,28.756900
3,2023-06-01 03:00:00,0,16.465381,4.362756,11.717233,1,1,2,19.707243,11,5.162315,1,0,13.77,89.01,3,0.610317,28.556674
4,2023-06-01 04:00:00,21,17.650720,5.727740,19.267657,2,34,3,22.953785,13,9.049066,2,0,14.27,81.00,4,0.609933,28.556675
5,2023-06-01 05:00:00,12,17.272565,3.986613,6.058799,0,0,2,19.074010,9,7.968235,2,0,15.06,80.10,5,0.609537,28.381483
6,2023-06-01 06:00:00,12,27.863739,3.245102,21.827980,2,45,1,17.939803,13,5.510599,1,0,16.10,88.10,6,0.609557,29.707961
7,2023-06-01 07:00:00,41,52.271777,5.813971,14.187327,1,24,3,21.181540,18,9.834054,3,0,17.31,98.50,7,0.609516,27.355346
8,2023-06-01 08:00:00,31,55.474211,4.728541,18.233220,1,44,2,18.413512,17,7.561134,2,0,18.60,99.07,8,0.609570,28.356455
9,2023-06-01 09:00:00,32,54.667023,3.414813,15.624900,2,35,2,21.713372,24,7.653581,3,29,19.89,87.30,9,0.609632,28.857011


In [11]:
# Save the result to a new CSV file
merged_dataframe.to_csv('optimizer/optimizer_input/parameters.csv', index=False)